In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

c:\Users\Dell\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Dell\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
c:\Users\Dell\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
df = pd.read_csv('schedulenets.csv')

In [3]:
df.head()

,updatedAt,Date,Time,block_1
0,2018-11-06 17:31:06,2018-11-06,17:31:06,372.000000
1,2018-11-05 12:53:55,2018-11-05,12:53:55,372.000000
2,2018-11-12 17:16:58,2018-11-12,17:16:58,373.000000
3,2018-11-11 16:58:16,2018-11-11,16:58:16,373.000000
4,2018-11-13 16:52:33,2018-11-13,16:52:33,412.874368


In [4]:

df['time'] = pd.to_datetime(df['updatedAt'])

df['Date'] = df['time'].dt.date
df['Time'] = df['time'].dt.strftime('%H:%M')
df['Voltage']=df['block_1']

df = df[[ 'Date', 'Time','Voltage']]
df

,Date,Time,Voltage
0,2018-11-06,17:31,372.000000
1,2018-11-05,12:53,372.000000
2,2018-11-12,17:16,373.000000
3,2018-11-11,16:58,373.000000
4,2018-11-13,16:52,412.874368
...,...,...,...
10538,2021-06-11,09:30,200.000000
10539,2021-06-11,04:27,546.000000
10540,2021-06-11,06:07,466.000000
10541,2021-06-11,08:30,546.000000


In [5]:
df['Time_Intervals'] = df['Time'].apply(lambda x: f"{x[:2]}:{int(x[3:]) // 15 * 15:02d} to {x[:2]}:{(int(x[3:]) // 15 + 1) * 15:02d}")
df['Average_Voltage'] = df.groupby('Time_Intervals')['Voltage'].transform('mean')

df = df.groupby(['Date', 'Time_Intervals']).agg({'Average_Voltage': 'mean'}).reset_index()

data = df[[ 'Date', 'Time_Intervals','Average_Voltage']]
# df.to_csv('voltage_with_Intervals.csv', index=False) 
data

,Date,Time_Intervals,Average_Voltage
0,2018-11-05,12:45 to 12:60,417.272665
1,2018-11-05,17:30 to 17:45,407.169412
2,2018-11-06,17:30 to 17:45,407.169412
3,2018-11-10,16:45 to 16:60,386.672865
4,2018-11-11,16:45 to 16:60,386.672865
...,...,...,...
6540,2021-06-11,04:15 to 04:30,391.285290
6541,2021-06-11,06:00 to 06:15,401.741830
6542,2021-06-11,08:30 to 08:45,401.910695
6543,2021-06-11,09:00 to 09:15,397.025748


In [6]:
# Data Preprocessing 
data['Date'] = pd.to_datetime(data['Date'])
data['Time_Intervals'] = data['Time_Intervals'].str.replace('60', '00', regex=True)
data['Hour'] = data['Time_Intervals'].str.split(':').str[0].str[-2:].astype(int)
data['Minute'] = data['Time_Intervals'].str.split(':').str[1].str[:2].astype(int)
data['Average_Voltage'] = MinMaxScaler().fit_transform(data['Average_Voltage'].values.reshape(-1, 1))
data

,Date,Time_Intervals,Average_Voltage,Hour,Minute
0,2018-11-05,12:45 to 12:00,0.621653,12,45
1,2018-11-05,17:30 to 17:45,0.554991,17,30
2,2018-11-06,17:30 to 17:45,0.554991,17,30
3,2018-11-10,16:45 to 16:00,0.419753,16,45
4,2018-11-11,16:45 to 16:00,0.419753,16,45
...,...,...,...,...,...
6540,2021-06-11,04:15 to 04:30,0.450186,4,15
6541,2021-06-11,06:00 to 06:15,0.519179,6,0
6542,2021-06-11,08:30 to 08:45,0.520293,8,30
6543,2021-06-11,09:00 to 09:15,0.488062,9,0


In [7]:
data.drop(['Time_Intervals'], axis=1, inplace=True)

In [8]:
n_steps = 3 
X, y = [], []

In [9]:
for i in range(n_steps, len(data)):
    X.append(data[['Hour', 'Minute', 'Average_Voltage']].values[i - n_steps:i])
    y.append(data['Average_Voltage'].values[i])

In [10]:
X, y = np.array(X), np.array(y)

In [11]:
X = X.reshape((X.shape[0], X.shape[1], X.shape[2]))

In [12]:
# Building the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(LSTM(units=50, return_sequences=True))
model.add(LSTM(units=50))
model.add(Dense(units=1))

In [13]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [14]:
# Training the model
model.fit(X, y, epochs=100, batch_size=32)

Epoch 1/100
205/205 [==============================] - 8s 8ms/step - loss: 0.0383
Epoch 2/100
205/205 [==============================] - 2s 8ms/step - loss: 0.0307
Epoch 3/100
205/205 [==============================] - 2s 8ms/step - loss: 0.0302
Epoch 4/100
205/205 [==============================] - 2s 8ms/step - loss: 0.0298
Epoch 5/100
205/205 [==============================] - 2s 8ms/step - loss: 0.0295
Epoch 6/100
205/205 [==============================] - 2s 8ms/step - loss: 0.0289
Epoch 7/100
205/205 [==============================] - 2s 8ms/step - loss: 0.0291
Epoch 8/100
205/205 [==============================] - 2s 8ms/step - loss: 0.0289
Epoch 9/100
205/205 [==============================] - 2s 8ms/step - loss: 0.0290
Epoch 10/100
205/205 [==============================] - 2s 8ms/step - loss: 0.0290
Epoch 11/100
205/205 [==============================] - 2s 8ms/step - loss: 0.0289
Epoch 12/100
205/205 [==============================] - 2s 8ms/step - loss: 0.0290
Epoch 13/100


In [15]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error,mean_squared_error

# Split the data into features (X) and target variable (y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model using the training set
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)

# Making predictions on the test set
predictions = model.predict(X_test)

scaler=StandardScaler
# Reverse scaling for predictions and actual values
predictions_original = scaler.inverse_transform(predictions.reshape(-1, 1))
y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1))

# Calculate regression metrics
mae = mean_absolute_error(y_test_original, predictions_original)
mse = mean_squared_error(y_test_original, predictions_original)
rmse = np.sqrt(mse)

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')

# Optionally, you can visualize the results
plt.figure(figsize=(12, 6))
plt.plot(y_test_original, label='Actual')
plt.plot(predictions_original, label='Predicted')
plt.legend()
plt.show()

41/41 [==============================] - 1s 5ms/step


TypeError: inverse_transform() missing 1 required positional argument: 'X'

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# Calculate accuracy using the provided formula
mean_value = y_test_original.mean()
accuracy = (1 - (rmse / mean_value)) * 100

print(f"Accuracy: {accuracy:.2f}%")


Accuracy: 92.82%
